In [5]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

In [6]:
# 드라이버 테스트 
driver = webdriver.Chrome()
driver.implicitly_wait(10) # 10초 대기

driver.get('https://google.co.kr')

time.sleep(5) # 5초 대기
driver.quit() # 웹 브라이저 종료 driver.close() 탭 종료

In [25]:
# 접속, 팝업삭제, 랭킹페이지 소스코드 가지고 오기
driver = webdriver.Chrome()
driver.implicitly_wait(10) # 10초 대기

url = 'https://www.musinsa.com/main/musinsa/ranking?storeCode=musinsa&sectionId=199&categoryCode=100000&period=MONTHLY'
driver.get(url)
time.sleep(3) # 3초 대기

try:
    popup = driver.find_element(By.CSS_SELECTOR, 'div._gateway-additional_3fnt1_52 button')
    popup.click()
    time.sleep(3)
except:
    print('팝업창 없음')

html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
driver.quit()

In [45]:
# 랭킹에 있는 url 가져오기
product_links = []

for link in soup.select('.sc-1m4cyao-1 a'):
    product_url = link.get('href')
    product_links.append(product_url)

df = pd.DataFrame(product_links, columns=['URL'])
df['review URL'] = df['URL'].str.replace('/app/goods/', '/review/goods/') #리뷰페이지로 바로가는 주소 확보

csv_filename = 'musinsa_crawling_linklist.csv'
df.to_csv(csv_filename, index=False)

In [46]:
df

,URL
0,https://www.musinsa.com/app/goods/1875663
1,https://www.musinsa.com/app/goods/3977452
2,https://www.musinsa.com/app/goods/1875664
3,https://www.musinsa.com/app/goods/4018731
4,https://www.musinsa.com/app/goods/3977488
...,...
195,https://www.musinsa.com/app/goods/4101470
196,https://www.musinsa.com/app/goods/3971202
197,https://www.musinsa.com/app/goods/4056085
198,https://www.musinsa.com/app/goods/4108244


In [49]:
# 한 페이지에서 html 소스코드 가져오기
driver = webdriver.Chrome()
driver.implicitly_wait(10) 

product_url = 'https://www.musinsa.com/products/1875663'
driver.get(product_url)
time.sleep(3)

html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
driver.quit()

In [66]:
# 한 페이지_상품후기더보기에서 html 소스코드 가져오기
driver = webdriver.Chrome()
driver.implicitly_wait(10) 

product_url = 'https://www.musinsa.com/review/goods/1875663'
driver.get(product_url)
time.sleep(3)

detail_html = driver.page_source
soup = BeautifulSoup(detail_html, 'lxml')
driver.quit()

In [ ]:
import pandas as pd
import re
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# 리뷰 수 추출 함수 정의
def extract_review_count(text):
    text = text.replace(',', '')
    match = re.search(r'\d+', text)
    if match:
        return int(match.group())
    else:
        return 0
        
# 리뷰 수집 함수 정의
def collect_reviews(product_url, product_id):
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    
    driver.get(product_url)
    time.sleep(3)

    detail_html = driver.page_source
    soup = BeautifulSoup(detail_html, 'lxml')

    count = soup.select('.text-title_18px_med')
    if count:
        review_text = count[0].text
        review_count = extract_review_count(review_text)
    else:
        review_count = 0

    # 리뷰수 100개 이상일 때 수집 진행
    if review_count >= 100:
        # 제목 추출
        title_element = soup.select_one('.sc-eulNPF')
        title = title_element.text if title_element else 'None'
        
        filtered_reviews = []
        collected_reviews = 0

        # 페이지 높이 초기화
        last_height = driver.execute_script("return document.body.scrollHeight")

        while collected_reviews < 1000:
            # 페이지 소스 가져오기 및 파싱
            detail_html = driver.page_source
            soup = BeautifulSoup(detail_html, 'lxml')

            # 리뷰와 별점을 각각 다른 클래스에서 수집
            review_elements = soup.select(".sc-bbSYpP p")  # 리뷰 텍스트가 있는 요소
            star_elements = soup.select('.text-body_14px_semi')  # 별점이 있는 요소

            # 리뷰 - 별점 같은 수량인지 확인
            min_length = min(len(review_elements), len(star_elements))
            
            for i in range(min_length):
                review_text = review_elements[i].text.strip()
                star = star_elements[i].text.strip()
                
                filtered_reviews.append({"Review": review_text, "Star": star})
                collected_reviews += 1
            
        
            # 1000개 리뷰 수집 시 종료
            if collected_reviews >= 1000:
                break
    
            # 페이지 끝까지 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # 스크롤 후 대기

            # 스크롤이 끝났을 시 종료 / 새로운 페이지 높이 계산
            new_height = driver.execute_script("return document.body.scrollHeight") # 높이비교
            if new_height == last_height:
                # 더 이상 스크롤할 내용이 없는 경우 종료
                print("더 이상 스크롤할 내용이 없으므로 종료합니다.")
                break
            last_height = new_height # 높이 업데이트
        
        driver.quit()
    
        # 수집된 리뷰, 별점을 DataFrame으로 변환하고 CSV 파일로 저장
        df_reviews = pd.DataFrame(filtered_reviews, columns=["Review", "Star"])
        csv_filename = f'reviews_{product_id}.csv'
        df_reviews.to_csv(csv_filename, index=False)
    
        print(f"총 수집된 리뷰 개수: {collected_reviews}")
        print(f"CSV 파일로 저장 완료: {csv_filename}")

        return {'Product ID': product_id, 'Title': title, 'Review Count': collected_reviews}

    else:
        print(f"{product_id}의 리뷰 수가 {review_count}개로 100개 미만이므로 수집하지 않습니다.")
    
    driver.quit()
    return None

df_links = pd.read_csv('musinsa_crawling_linklist.csv')

# Product ID 추출
df_links['Product ID'] = df_links['review URL'].str.extract(r'/goods/(\d+)$')

summary_list = []

# 링크에 대해 리뷰 수집
for index, row in df_links.iterrows():
    summary_data = collect_reviews(row['review URL'], row['Product ID'])
    if summary_data:
        summary_list.append(summary_data)

# 전체 summary_df 생성
summary_df = pd.DataFrame(summary_list)

# 전체 summary_df를 CSV 파일로 저장
summary_df.to_csv('overall_summary.csv', index=False)
print(f"전체 요약 CSV 파일로 저장 완료: overall_summary.csv")

In [3]:
summary_df.info()

,URL,review URL
0,https://www.musinsa.com/app/goods/1875663,https://www.musinsa.com/review/goods/1875663
1,https://www.musinsa.com/app/goods/3977452,https://www.musinsa.com/review/goods/3977452
2,https://www.musinsa.com/app/goods/1875664,https://www.musinsa.com/review/goods/1875664
3,https://www.musinsa.com/app/goods/4018731,https://www.musinsa.com/review/goods/4018731
4,https://www.musinsa.com/app/goods/3977488,https://www.musinsa.com/review/goods/3977488
...,...,...
195,https://www.musinsa.com/app/goods/4101470,https://www.musinsa.com/review/goods/4101470
196,https://www.musinsa.com/app/goods/3971202,https://www.musinsa.com/review/goods/3971202
197,https://www.musinsa.com/app/goods/4056085,https://www.musinsa.com/review/goods/4056085
198,https://www.musinsa.com/app/goods/4108244,https://www.musinsa.com/review/goods/4108244


In [ ]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time

df = pd.read_csv('overall_summary.csv')

brand_names = []

for product_id in df['Product ID']:
    driver = webdriver.Chrome()
    driver.implicitly_wait(10) 

    product_url = f'https://www.musinsa.com/products/{product_id}'
    driver.get(product_url)
    time.sleep(3)

    # 페이지 소스 가져오기 및 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    driver.quit()

    # 브랜드 이름 추출
    brandname_elements = soup.select('.text-sm')
    if brandname_elements:
        brand_name = brandname_elements[0].text.strip()
    else:
        brand_name = "Unknown"

    # 브랜드 이름을 리스트에 추가
    brand_names.append(brand_name)

# 추출한 브랜드 이름을 새로운 열로 추가
df['Brand Name'] = brand_names

# 결과를 CSV 파일로 저장
df.to_csv('overall_summary_with_brand.csv', index=False)
print(f"브랜드 이름이 추가된 CSV 파일로 저장 완료: overall_summary_with_brand.csv")

In [ ]:
df['Brand Name'].isna().sum()

In [ ]:
# Title을 다시 크롤링하는 함수 정의
def update_title(product_id):
    driver = webdriver.Chrome()
    driver.implicitly_wait(10) 

    try:
        product_url = f'https://www.musinsa.com/products/{product_id}'
        driver.get(product_url)
        time.sleep(3)

        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        title_element = soup.select_one('.text-lg')
        if title_element:
            title = title_element.text.strip()
            if not title:  # title이 비어있는 경우
                title = np.nan
        else:
            title = np.nan  # title 요소가 없을 때
    except Exception as e:
        print(f"오류 발생: {e}")
        title = np.nan  # 에러가 발생할 경우에도 NaN으로 처리
    finally:
        driver.quit()

    return title

# CSV 파일에서 데이터 읽기
df = pd.read_csv('overall_summary_with_brand.csv')

# NaN 값을 가진 Title 열만 선택
nan_titles = df[df['Title'].isna()]

# NaN 값을 가진 각 Product ID에 대해 다시 크롤링
for index, row in nan_titles.iterrows():
    product_id = row['Product ID']
    
    # 새로운 title 가져오기
    new_title = update_title(product_id)
    
    # DataFrame에 업데이트
    df.at[index, 'Title'] = new_title

# 업데이트된 DataFrame을 CSV 파일로 저장
df.to_csv('overall_summary_updated.csv', index=False)
print(f"업데이트된 CSV 파일로 저장 완료: overall_summary_updated.csv")